GLUE sets: model will be trained on eval set, so you shouldn't also test on the eval set. The problem is that the labels are withheld for the test set. 
Start with SNLI. MultiNLI is a later option too. As is rotten_tomatoes. 
* Victim model performance on dataset train, valid, test set. (done, written code to measure it)
* Create new paraphrased valid + test datasets (done a preliminary version on the valid set) 
* Measure victim model performance on paraphrased datasets (done. on vanilla valid set is about 87% accuracy. generating 16 paraphrases (i.e. not many) and evaluating performance on all of them, we get ~75% accuracy)
* Get document embeddings of original and paraphrased and compare (done)
  * https://github.com/UKPLab/sentence-transformers
* Write a simple way to measure paraphrase quality (done) 
* Construct reward function 


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch 
from torch.utils.data import DataLoader
from datasets import load_dataset, load_metric
import datasets, transformers
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, AutoTokenizer
from pprint import pprint
import numpy as np, pandas as pd
from utils import *   # local script 
import pyarrow
from sentence_transformers import SentenceTransformer, util
from IPython.core.debugger import set_trace
from GPUtil import showUtilization
import seaborn as sns

path_cache = './cache/'
path_results = "./results/"

seed = 420
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
devicenum = torch.cuda.current_device() if device.type == 'cuda' else -1
n_wkrs = 4 * torch.cuda.device_count()
batch_size = 64
pd.set_option("display.max_colwidth", 400)

In [3]:
# Paraphrase model (para)
para_name = "tuner007/pegasus_paraphrase"
para_tokenizer = AutoTokenizer.from_pretrained(para_name)
para_model = AutoModelForSeq2SeqLM.from_pretrained(para_name).to(device)

In [4]:
# Victim Model (VM)
vm_name = "textattack/distilbert-base-cased-snli"
vm_tokenizer = AutoTokenizer.from_pretrained(vm_name)
vm_model = AutoModelForSequenceClassification.from_pretrained(vm_name).to(device)
vm_idx2lbl = vm_model.config.id2label
vm_lbl2idx = vm_model.config.label2id
vm_num_labels = vm_model.num_labels

In [5]:
# Semantic Similarity model 
embedding_model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [6]:
dataset = load_dataset("snli")
train,valid,test = dataset['train'],dataset['validation'],dataset['test']

label_cname = 'label'
remove_minus1_labels = lambda x: x[label_cname] != -1
train = train.filter(remove_minus1_labels)
valid = valid.filter(remove_minus1_labels)
test = test.filter(remove_minus1_labels)

# make sure that all datasets have the same number of labels as what the victim model predicts
assert train.features[label_cname].num_classes == vm_num_labels
assert valid.features[label_cname].num_classes == vm_num_labels
assert test.features[ label_cname].num_classes == vm_num_labels

train_dl = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=n_wkrs)
valid_dl = DataLoader(valid, batch_size=batch_size, shuffle=True, num_workers=n_wkrs)
test_dl = DataLoader( test,  batch_size=batch_size, shuffle=True, num_workers=n_wkrs)

Reusing dataset snli (/data/tproth/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


In [7]:
def get_paraphrases(input_text,num_return_sequences,num_beams, num_beam_groups=1,diversity_penalty=0):
    batch = para_tokenizer(input_text,truncation=True,padding='longest', return_tensors="pt").to(device)
    translated = para_model.generate(**batch,num_beams=num_beams, num_return_sequences=num_return_sequences, 
                                   temperature=1.5, num_beam_groups=num_beam_groups, diversity_penalty=diversity_penalty)
    tgt_text = para_tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

def gen_hypothesis_paraphrases(x, n_seed_seqs=32): 
    """keep n_seed_seqs at 4,8,16,32,64 etc"""
    # TODO: figure out how to batch this. 
    if n_seed_seqs % 4 != 0: raise ValueError("keep n_seed_seqs divisible by 4 for now")
    n = n_seed_seqs/2
    #low diversity (ld) paraphrases 
    ld_l = get_paraphrases(x['hypothesis'],num_return_sequences=int(n),
                            num_beams=int(n))
    #high diversity (hd) paraphrases. We can use num_beam_groups and diversity_penalty as hyperparameters. 
    hd_l =  get_paraphrases(x['hypothesis'],num_return_sequences=int(n),
                            num_beams=int(n), num_beam_groups=int(n),diversity_penalty=50002.5)
    l = ld_l + hd_l 
    x['hypothesis_paraphrases'] = l #TODO: change to list(set(l))             
    return x 


In [8]:
n_seed_seqs = 48
fname = path_cache + 'valid_small_'+ str(n_seed_seqs)
if os.path.exists(fname):  # simple caching
    valid_small = datasets.load_from_disk(fname)
else:
    valid_small = valid.shard(20, 0, contiguous=True)
    valid_small = valid_small.map(lambda x: gen_hypothesis_paraphrases(x, n_seed_seqs=n_seed_seqs),
                  batched=False)
    valid_small.save_to_disk(fname)
    
    

In [9]:
# Create a new dataset by repeating all other fields to be same length as number of paraphrases. 
def create_paraphrase_dataset(batch): 
    """Repeat the other fields to be the same length as the number of paraphrases."""    
    n_premises = len(batch['premise'])
    paraphrases,hyp,prem,labels=[],[],[],[]
    d1 = dict()
    def rep_entry(x): return [x for o in range(n_paraphrases)]
    for p,h,l,hp in zip(batch['premise'], batch['hypothesis'], batch['label'], batch['hypothesis_paraphrases']):
        n_paraphrases = len(hp)
        paraphrases +=    hp
        hyp         +=    rep_entry(h)
        prem        +=    rep_entry(p)
        labels      +=    rep_entry(l)
    return {
        'hypothesis': hyp,
        'hypothesis_paraphrases': paraphrases,
        'premise':prem,
        'label':labels 
       }

In [10]:
fname = path_cache + 'valid_small_paraphrases_' + str(n_seed_seqs)
if os.path.exists(fname):     
    valid_small_paraphrases = datasets.load_from_disk(fname)
else:
    # Need to call this with batched=True to work. 
    valid_small_paraphrases = valid_small.map(create_paraphrase_dataset,batched=True)
    valid_small_paraphrases.save_to_disk(fname)


In [11]:
def get_vm_scores(): 
    """very hacky procedure to generate victim model scores """
    # Get preds and accuracy on the paraphrase dataset
    print("Getting victim model scores.")
    some_dl = DataLoader(valid_small_paraphrases, batch_size=batch_size, shuffle=False, 
                         num_workers=n_wkrs, pin_memory=True)
    dl = some_dl
    metric = load_metric('accuracy')
    para_probs_l,orig_probs_l = [], []
    assert vm_model.training == False  # checks that model is in eval mode 
    #monitor = Monitor(2)  # track GPU usage and memory
    with torch.no_grad():
        for i, data in enumerate(dl): 
            if i % 50 == 0 : print(i, "out of", len(dl))
            labels,premise = data['label'].to(device),data["premise"]
            paraphrases,orig = data["hypothesis_paraphrases"],data["hypothesis"]

            # predictions for original
            inputs = vm_tokenizer(premise,orig,padding=True,truncation=True, return_tensors="pt")
            inputs.to(device)
            outputs = vm_model(**inputs, labels=labels)
            probs = outputs.logits.softmax(1)
            preds = probs.argmax(1)
            orig_probs_l.append(probs.cpu())  

            # predictions for paraphrases
            inputs = vm_tokenizer(premise,paraphrases, padding=True,truncation=True, return_tensors="pt")
            inputs.to(device)
            outputs = vm_model(**inputs, labels=labels)
            probs = outputs.logits.softmax(1)
            preds = probs.argmax(1)
            para_probs_l.append(probs.cpu())
            metric.add_batch(predictions=preds, references=labels)

    orig_probs_t, para_probs_t = torch.cat(orig_probs_l),torch.cat(para_probs_l)
    #monitor.stop()

    # bit of a hack, i'm sure there's a native pytorch function for this but I couldn't find it
    vm_para_scores = torch.tensor([r[idx] for idx,r in zip(valid_small_paraphrases['label'],para_probs_t)])
    vm_orig_scores = torch.tensor([r[idx] for idx,r in zip(valid_small_paraphrases['label'],orig_probs_t)])
    return para_probs_t, orig_probs_t
    

def generate_sim_scores(): 
    """Function to just loop and generate sim scores for each input"""
    print("Getting similarity scores")
    sim_score_l = []
    for i, data in enumerate(valid_small): 
        if i % 50 == 0 : print(i, "out of", len(valid_small))
        orig, para = data['hypothesis'], data['hypothesis_paraphrases']
        orig_emb,para_emb  = embedding_model.encode(orig),embedding_model.encode(para)
        cos_sim = util.cos_sim(orig_emb,para_emb)[0]
        sim_score_l.append(cos_sim)
    sim_score_t = torch.cat(sim_score_l)
    return sim_score_t


In [12]:
# Generate results dataframe 
fname = path_cache + 'results_df' + str(n_seed_seqs) + "_1.csv"
if os.path.exists(fname):
    results_df = pd.read_csv(fname)
else: 
    sim_score_t = generate_sim_scores()
    para_probs_t, orig_probs_t = get_vm_scores()
    vm_para_scores = torch.tensor([r[idx] for idx,r in zip(valid_small_paraphrases['label'],para_probs_t)])
    vm_orig_scores = torch.tensor([r[idx] for idx,r in zip(valid_small_paraphrases['label'],orig_probs_t)])
    
    results_df = pd.DataFrame({'premise': valid_small_paraphrases['premise'],
                  'orig': valid_small_paraphrases['hypothesis'],
                  'para': valid_small_paraphrases['hypothesis_paraphrases'],
                  'sim_score': sim_score_t,
                  'label_true': valid_small_paraphrases['label'], 
                  'label_vm_orig': orig_probs_t.argmax(1),
                  'label_vm_para': para_probs_t.argmax(1),
                  'vm_orig_truelabel': vm_orig_scores,             
                  'vm_para_truelabel': vm_para_scores,
                  'vm_truelabel_change': vm_orig_scores - vm_para_scores,
                  'vm_orig_class0': orig_probs_t[:,0], 
                  'vm_orig_class1': orig_probs_t[:,1], 
                  'vm_orig_class2': orig_probs_t[:,2],  
                  'vm_para_class0': para_probs_t[:,0], 
                  'vm_para_class1': para_probs_t[:,1], 
                  'vm_para_class2': para_probs_t[:,2]     
                  })
    results_df['vm_truelabel_change_X_sim_score'] = results_df['vm_truelabel_change'] * results_df['sim_score']
    results_df.to_csv(fname, index_label = 'idx')

In [13]:
results_df

,idx,premise,orig,para,sim_score,label_true,label_vm_orig,label_vm_para,vm_orig_truelabel,vm_para_truelabel,vm_truelabel_change,vm_orig_class0,vm_orig_class1,vm_orig_class2,vm_para_class0,vm_para_class1,vm_para_class2,vm_truelabel_change_X_sim_score
0,0,Two women are embracing while holding to go packages.,The sisters are hugging goodbye while holding to go packages after just eating lunch.,The sisters are hugging goodbye after eating lunch.,0.916945,1,1,1,0.996048,0.994097,0.001952,0.002556,0.996048,0.001395,0.002258,0.994097,0.003646,0.001790
1,1,Two women are embracing while holding to go packages.,The sisters are hugging goodbye while holding to go packages after just eating lunch.,The sisters are hugging goodbye as they hold to go packages.,0.820021,1,1,1,0.996048,0.991314,0.004734,0.002556,0.996048,0.001395,0.006501,0.991314,0.002185,0.003882
2,2,Two women are embracing while holding to go packages.,The sisters are hugging goodbye while holding to go packages after just eating lunch.,The sisters are holding onto packages after eating lunch.,0.856539,1,1,1,0.996048,0.994749,0.001299,0.002556,0.996048,0.001395,0.002427,0.994749,0.002824,0.001113
3,3,Two women are embracing while holding to go packages.,The sisters are hugging goodbye while holding to go packages after just eating lunch.,The sisters are hugging and holding onto packages after eating lunch.,0.905042,1,1,1,0.996048,0.995355,0.000693,0.002556,0.996048,0.001395,0.002345,0.995355,0.002300,0.000627
4,4,Two women are embracing while holding to go packages.,The sisters are hugging goodbye while holding to go packages after just eating lunch.,The sisters are holding packages after eating lunch.,0.858632,1,1,1,0.996048,0.995044,0.001004,0.002556,0.996048,0.001395,0.002618,0.995044,0.002338,0.000862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23659,23659,"Nine women in white robes with hoods walk on plush, green grass.",The women are wearing flip flops.,These Women have flipped flop shoes on,0.726975,1,2,2,0.100099,0.222727,-0.122628,0.000503,0.100099,0.899398,0.001628,0.222727,0.775645,-0.089148
23660,23660,"Nine women in white robes with hoods walk on plush, green grass.",The women are wearing flip flops.,Woman withflops,0.678554,1,2,2,0.100099,0.112493,-0.012394,0.000503,0.100099,0.899398,0.393144,0.112493,0.494363,-0.008410
23661,23661,"Nine women in white robes with hoods walk on plush, green grass.",The women are wearing flip flops.,Woman withflops on them,0.691026,1,2,2,0.100099,0.168825,-0.068726,0.000503,0.100099,0.899398,0.045529,0.168825,0.785646,-0.047492
23662,23662,"Nine women in white robes with hoods walk on plush, green grass.",The women are wearing flip flops.,Three girls were dressed inflip flops,0.525657,1,2,2,0.100099,0.147149,-0.047050,0.000503,0.100099,0.899398,0.032746,0.147149,0.820106,-0.024732


In [14]:
tmp = results_df.sort_values('vm_change_X_sim_score', ascending=False)

KeyError: 'vm_change_X_sim_score'

In [ ]:
tmp[0:40]

In [ ]:
results_df.iloc[22563:]

In [ ]:
grouped_df = results_df.groupby(['premise', 'orig'])

In [ ]:
results_df.sort_values()

In [ ]:
results_df.iloc[2000:2040,:]

## Archive 

In [ ]:
# # calculates performance of victim model on a dataloader 
# dl = valid_dl
# metric = load_metric('accuracy')
# for i, data in enumerate(dl): 
#     if i % 10 == 0 : print(i, "out of", len(dl)) 
#     labels,premise,hypothesis = data['label'].to(device),data["premise"],data["hypothesis"]
#     inputs = vm_tokenizer(premise,hypothesis, padding=True,truncation=True, return_tensors="pt")
#     inputs.to(device)
#     outputs = vm_model(**inputs, labels=labels)
#     probs = outputs.logits.softmax(1)
#     preds = probs.argmax(1)
#     metric.add_batch(predictions=preds, references=labels)

# metric.compute()


In [ ]:
# # Score semantic similarity with cross encoders
# from sentence_transformers.cross_encoder import CrossEncoder
# cross_encoder= CrossEncoder('cross-encoder/quora-distilroberta-base')
# i =11
# data = valid_small[i]
# orig, para = data['hypothesis'], data['hypothesis_paraphrases']
# orig_rep = [orig for i in range(len(para))]
# pairs = list(zip(orig_rep,para))
# scores = cross_encoder.predict(pairs)
# results_df = pd.DataFrame({'pairs':pairs, 'para': para,'score': cos_sim})
# print(orig)
# results_df.sort_values('score', ascending=False)

In [ ]:
# # with sentence transformers
# valid_small_dl = DataLoader(valid_small, batch_size=4, shuffle=False, 
#                      num_workers=n_wkrs, pin_memory=True)
# sim_score_l = []
# for i, data in enumerate(valid_small_dl): 
#     pass
#     orig, para = data['hypothesis'], data['hypothesis_paraphrases']
#     orig_emb,para_emb  = embedding_model.encode(orig),embedding_model.encode(para)
# #     cos_sim = util.cos_sim(orig_emb,para_emb)[0]
# #     results_df = pd.DataFrame({'para': para,'score': cos_sim})
# #     print(orig)
# #     results_df.sort_values('score', ascending=False)